In [ ]:
!pip install langchain langchain-openai langchain-pinecone langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 12.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

# pinecone 저장

In [ ]:
import json
from langchain_core.documents import Document

def load_jsonl_to_documents(jsonl_path):
    documents = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            content = data.get("page_content", "")
            metadata = data.get("metadata", {})
            documents.append(Document(page_content=content, metadata=metadata))
    return documents

documents = load_jsonl_to_documents("documents1.jsonl")
print(f"{len(documents)}개의 문서를 불러왔습니다.")
print(documents[0])

200개의 문서를 불러왔습니다.
page_content=''11종 혼합유산균'은 일동바이오사이언스(주)에서 제조한 건강기능식품 원료입니다. 이 제품은 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음 등의 기능이 있습니다. 기본 기준은 다음과 같습니다: 1) 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말
2) 프로바이오틱스 수(표시량 300,000,000,000(3,000억) CFU/g) : 표시량 이상
3) 대장균군 : 음성
4) 납 : 1)0 mg/kg 이하
5) 카드뮴 : 0)3 mg/kg 이하 보관 조건은 냉장조건입니다. 소비 및 유통기한은 제조일로부터 24개월까지입니다. 주 용도는 '건강기능식품 원료로 사용'입니다. 소비 및 유통기준 '냉장조건'입니다.' metadata={'PRDUCT': ' 11종 혼합유산균', 'ENTRPS': '일동바이오사이언스(주)', 'DISTB_PD': '제조일로부터 24개월까지'}


In [ ]:
# 문서로부터 벡터스토어 생성
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

PINECONE_INDEX_NAME = 'product'

# 임베딩 모델
embeddings = OpenAIEmbeddings(model=os.environ['OPENAI_EMBEDDING_MODEL'])

vector_store = PineconeVectorStore.from_documents(
    documents,
    embedding=embeddings,
    index_name='product'
)

In [ ]:
# 존재하는 인덱스에 접근/검색
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
vector_store = PineconeVectorStore(
    index=pc.Index('product'),
    embedding=embeddings
)

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)
retriever.invoke('11종 혼합유산균 섭취 시 주의사항은?')

[Document(id='3c28b35a-99d7-4470-a948-55bb8d47615b', metadata={'DISTB_PD': '18개월', 'ENTRPS': '(주)메디오젠', 'PRDUCT': '11종혼합유산균 아이(I)'}, page_content="'11종혼합유산균 아이(I)'은 (주)메디오젠에서 제조한 건강기능식품 원료입니다. 이 제품은 1)유산균 증식 및 유해균 억제에 도움을 줄 수 있음 \n2)배변활동 원활에 도움을 줄 수 있음 등의 기능이 있습니다. 기본 기준은 다음과 같습니다: 1)성상 : 이미,이취가 없는 황색 ~황갈색의 분말\n2)대장균군 : 음성\n3)프로바이오틱스 수 : 1)0 X 10^11cfu/g 이상 보관 조건은 직사광선 및 고온다습한 곳을 피하여 냉암소에서 보관하고 유통중에 흡습되지 않게 파손에 주의입니다. 소비 및 유통기한은 18개월입니다. 주 용도는 '건강기능식품원료용 또는 일반식품의 원료로 적당량 사용한다'입니다. 소비 및 유통기준 '직사광선 및 고온다습한 곳을 피하여 냉암소에서 보관하고 유통중에 흡습되지 않게 파손에 주의'입니다."),
 Document(id='0ddeca54-e057-4adb-a9d4-ff547a4fcb3d', metadata={'DISTB_PD': '제조일로부터 24개월까지', 'ENTRPS': '(주)종근당바이오', 'PRDUCT': '11종 혼합유산균 300B (전량수출용)'}, page_content="'11종 혼합유산균 300B (전량수출용)'은 (주)종근당바이오에서 제조한 건강기능식품 원료입니다. 이 제품은 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음 등의 기능이 있습니다. 기본 기준은 다음과 같습니다: 1) 성상 : 고유의 향미가 있고 이미, 이취가 없는 흰 노랑의 분말\n2) 프로바이오틱스 수 : 표시량[100,000,000,000(1,000억) CFU/g] 이상\n3) 대장균군 : 음성\n4) 납(mg/kg) : 1)0 이하\n5) 카드뮴(mg/kg) :

"프로바이오틱스 함량이 높은 제품은?"

"냉장보관해야 하는 유산균 제품은?"

"씨티씨바이오 제품은 어떤 특징이 있어?"

"12종혼합유산균 제트(z) 보관법 알려줘"

In [ ]:
# 사용자 질문
query = "요즘 면역력이 약해진 것 같고 자주 피곤해요"

# 연관 문서 검색
retrieved = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved])

# RAG 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 문서를 참고하여 사용자의 건강 고민, 증상, 목표(예: 피로 회복, 장 건강, 눈 피로, 면역력 향상 등)를 파악한 뒤, 적합한 성분과 그 성분이 포함된 건강기능식품을 추천해주고, 섭취 팁, 주의사항, 보관법 등 실질적인 정보도 제공합니다."),
    ("human", "{context}\n\n질문: {query}")
])

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=1)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

# 실행
response = chain.invoke({"context": context, "query": query})
print(response)

면역력이 약해지고 자주 피로를 느끼는 경우, 체내 면역 기능을 강화하고 피로 회복에 도움을 줄 수 있는 성분이 포함된 건강기능식품 섭취를 권장합니다. 특히, 프로바이오틱스(유산균)와 비타민 D, 아연은 면역력 향상과 피로 회복에 효과적입니다.

추천하는 제품 및 성분:
1. **'100억 생유산균 + 면역플러스'**  
   - **주요 기능:** 유산균 증식, 유해균 억제, 장 건강, 면역력 증진  
   - **추천 이유:** 프로바이오틱스 10억 CFU 이상이 장내 환경을 개선하여 면역력을 높여주며, 비타민 D와 아연도 함유돼 있어 전반적인 면역력 강화와 피로 회복에 도움을 줄 수 있습니다. 이 제품은 하루 1회 섭취로 편리하게 면역 건강을 챙길 수 있습니다.

2. **'10억 생유산균 면역비타민D'**  
   - **주요 기능:** 유산균 증식과 유해균 억제, 비타민 D와 아연 함유로 면역력·뼈 건강 증진  
   - **추천 이유:** 아연과 비타민 D는 면역 기능을 지원하며 골격 건강에도 도움을 줍니다. 이 제품도 1일 1회 섭취로 간편하며, 유산균도 10억 CFU 포함돼 있어 장 건강과 면역력 강화를 동시에 할 수 있습니다.

**섭취 팁:**  
- 제품별로 권장 섭취량(예: 하루 1포 또는 5g)을 지켜 주세요.  
- 공복 또는 식사 후 섭취하는 것도 장 건강과 흡수에 도움을 줄 수 있습니다.  
- 꾸준히 섭취하는 것이 면역력 강화에 효과적입니다.

**주의사항:**  
- 제품에 포함된 성분 알레르기 유무를 확인하세요.  
- 섭취량을 초과하지 않도록 주의하세요.  
- 임신 또는 특정 질환이 있을 경우 전문가와 상담 후 섭취하세요.

**보관법:**  
- 직사광선 및 고온다습한 곳을 피해 서늘하고 건조한 곳에 보관하세요.  
- 어린이의 손이 닿지 않는 곳에 보관하세요.  
- 유통기한 내에 사용하세요.

이와 같이 장 건강과 면역력 강화를 위한 프로바이오틱스와 비타민D, 아연이 함유된 제품으로 건강을 챙기시기를 추천드립니다.


In [ ]:
# 사용자 질문 → 검색된 문맥 → LLM 응답 생성 함수
def get_rag_response(query: str, k: int = 5) -> str:
    docs = vector_store.similarity_search(query, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    response = chain.invoke({"context": context, "query": query})
    return response

print(get_rag_response("냉장보관해야 하는 유산균 제품은?"))

냉장보관이 권장되는 유산균 제품은 **'16종맘스유산균'**입니다.  
이 제품은 실온에서도 보관 가능하지만, **냉암소(냉장) 보관을 권장**하고 있어 신선도를 유지하기 위해 냉장 보관이 좋습니다.  

### 요약:
- **추천 제품:** 16종맘스유산균
- **보관 조건:** 실온(냉암소보관 권장) — 즉, 냉장보관이 가장 적합
- **기타 참고:** 냉장 보관 시 유산균의 활성도가 유지되고, 유통기한 내에 제품의 품질을 유지하는 데 도움을 줍니다.

### 섭취 팁
- 냉장 보관 시 제품의 품질과 유산균 생존율이 높아지므로, 반드시 개봉 후에는 냉장고에 넣어 보관하세요.
- 섭취 시에는 깨끗한 도구를 사용하고, 정해진 섭취량(예: 1포 또는 제조 지침)을 지키세요.

### 주의사항
- 직사광선이나 습기를 피해 서늘하고 건조한 곳에 보관하세요.
- 개봉 후에는 가능한 빠른 시일 내 섭취하는 것이 좋으며, 냉장 보관이 권장됩니다.
- 유산균 제품은 열이나 습도에 민감하므로, 보관 상태를 잘 유지하는 것이 중요합니다.

이외에도, 유산균 제품마다 보관 조건이 다를 수 있으니, 제품 포장이나 라벨에 명시된 보관 지침을 꼭 따라 주세요.


# 질문 - 그냥 llm..

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', """
페르소나: 당신은 영양학과 기능성 식품에 대한 전문 지식을 가진 건강기능식품 컨설턴트입니다. 다양한 기능성 원료(프로바이오틱스, 오메가3, 루테인 등)와 그 효과(면역력, 장 건강, 눈 건강 등)에 대한 최신 정보를 숙지하고 있으며, 사용자의 건강 고민에 맞는 성분과 제품을 친절하게 추천해줍니다.

역할: 사용자의 건강 고민, 증상, 목표(예: 피로 회복, 장 건강, 눈 피로, 면역력 향상 등)를 파악한 뒤, 적합한 성분과 그 성분이 포함된 건강기능식품을 추천합니다. 과학적 근거와 함께 섭취 팁, 주의사항, 보관법 등 실질적인 정보도 제공합니다. 초보자도 이해할 수 있게 설명하면서도 신뢰감 있는 전문성을 유지합니다.

예시:
- 사용자가 '요즘 배변이 잘 안 되고 더부룩해요'라고 하면, 프로바이오틱스와 프리바이오틱스가 포함된 제품을 추천하고 그 차이와 기능을 설명합니다.
- 사용자가 '눈이 자주 피로하고 침침해요'라고 하면, 루테인, 아스타잔틴, 비타민A 등이 포함된 성분을 소개하며 어떤 점이 도움이 되는지 알려줍니다.
- 사용자가 '스트레스를 많이 받아요'라고 하면, 테아닌, 마그네슘 등 스트레스 완화에 도움을 줄 수 있는 성분들을 소개합니다.
"""),
    ('human', """
다음과 같은 고민을 가진 사람에게 적절한 건강기능식품을 추천해주세요. (한국말로 상세히 답변해주세요.)

고민 또는 증상: {query}
""")
])

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=1)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

# 예시 호출
print(chain.invoke({'query': '요즘 면역력이 약해진 것 같고 자주 피곤해요'}))


안녕하세요! 요즘 면역력이 약해지고 자주 피곤하시다고 하셨군요. 이런 경우에는 면역력 향상과 피로 회복에 도움을 줄 수 있는 성분들이 포함된 건강기능식품이 좋습니다. 좀 더 구체적으로 설명드리자면:

### 추천 성분 및 효과
1. **비타민 C**: 강력한 항산화제로서 면역 세포의 활성화를 도와줍니다. 또한 피로 회복에 도움을 줄 수 있습니다.
2. **아연(zinc)**: 면역 기능 유지에 중요한 역할을 하며, 감염 예방과 회복을 돕습니다.
3. **프로바이오틱스(유산균)**: 장 건강과 면역력 강화에 기여합니다. 장이 건강하면 자연스럽게 면역 시스템도 강해집니다.
4. **비타민 D**: 면역 세포의 기능을 조절하며, 부족 시 감염에 대한 저항력이 떨어질 수 있기 때문에 중요합니다.
5. **홍삼 또는 인삼 추출물**: 스트레스 완화와 피로 회복, 면역력 증진에 효과적입니다.
6. **오메가-3( EPA와 DHA)**: 염증을 조절하고 면역 기능을 지원하는 역할을 합니다.

### 추천 상품 유형
이 성분들을 모두 포함한 종합 면역증진 영양제나, 일부만 조합된 제품을 찾는 것이 효과적입니다. 예를 들어:
- **면역력 강화용 종합 영양제**: 비타민 C, D, 아연, 프로바이오틱스, 홍삼 등이 함께 포함된 제품
- **개별 성분 집중 제품**: 비타민 D 또는 프로바이오틱스 단일 제품도 고려할 수 있습니다.

### 섭취 시 주의사항
- 제품의 성분과 함량을 확인하여 적합한 용량을 섭취하세요.
- 평소에 균형 잡힌 식단과 충분한 수면, 적절한 운동도 병행하는 것이 중요합니다.
- 만약 만성 질환이 있거나 의약품을 복용 중이시면, 전문가와 상담 후 섭취하세요.

### 보관법
직사광선과 고온 다습한 장소를 피하고, 뚜껑을 꼭 닫아 통풍이 잘 되는 서늘한 곳에 보관하세요.

면역력이 떨어지거나 피로감을 자주 느끼실 때는, 꾸준히 섭취하시면서 건강한 라이프스타일을 실천하는 것이 가장 중요합니다. 도움이 필요하시면 언제든 말씀해 주세요!
